## Still under development, not ready for student use

In [ ]:
# See install in M2-8a

In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

In [ ]:
#from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langchain_community.document_loaders import WebBaseLoader
url = "https://en.wikipedia.org/wiki/Elon_Musk"
loader = WebBaseLoader(url)
# Download the webpage into a variable
documents = loader.load()
# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Create the text splitter
text_splitter = RecursiveCharacterTextSplitter()
docs = text_splitter.split_documents(documents)
# Create embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
# Vector db
from langchain_community.vectorstores import FAISS
vector_db = FAISS.from_documents(docs, embeddings)
print(vector_db.index.ntotal)

62


In [8]:
retriever = vector_db.as_retriever()

In [9]:
# Model
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(api_key=openai_api_key)

In [17]:
from langchain_core.output_parsers import StrOutputParser

# This parser takes the AIMessage reutrned form the LLM and converts it to a string
output_parser = StrOutputParser()

In [18]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
my_prompt = ChatPromptTemplate.from_template(template)

In [19]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [14]:
#format_docs(docs)

In [21]:
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | my_prompt
    | openai_llm
    | StrOutputParser()
)

In [23]:
chain.invoke("Why is Elon so contriversal?")

'Elon Musk is controversial due to his outspoken views on various issues such as artificial intelligence, climate change, politics, and technology. His statements and actions, including his use of social media, have sparked controversy and criticism from various groups and individuals.'

In [ ]:
query = "Where was Elon born?"
docs = vector_db.similarity_search(query)

In [ ]:
#print(docs[0].page_content)

In [ ]:
retriever = vector_db.as_retriever()
docs = retriever.invoke(query)

In [ ]:
print(docs[0].page_content)

In [ ]:
# Retrive source material
#
from langchain_community.document_loaders import WebBaseLoader

#loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

url = "https://en.wikipedia.org/wiki/Elon_Musk"

loader = WebBaseLoader(url)
# Download the webpage into a variable
webpage_source = loader.load()

In [ ]:
type(webpage_source)
# Dump the string
#len(webpage_source[0].page_content)
word_count = len(webpage_source[0].page_content.split())
print(word_count)

In [ ]:
#%pip install langchain_openai
#%pip show langchain_openai
#%pip show typing-extensions

In [ ]:
# %pip uninstall typing_extensions -y
# %pip install typing_extensions
#%pip install --force-reinstall typing-extensions==4.5
#pip install --force-reinstall openai==1.8

In [ ]:
# from langchain_openai import ChatOpenAI
# openai_llm = ChatOpenAI(api_key=openai_api_key)

In [ ]:
# Vector store
#%pip install faiss-cpu
#%pip show faiss-cpu

In [ ]:
#from langchain_community.vectorstores import FAISS

In [ ]:
# Split the webpage_source into chunks
#
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Create the text splitter
text_splitter = RecursiveCharacterTextSplitter()
#dir(text_splitter)
text_splitter.__dict__
# Split the webpage_source into LangChain document

split_docs = text_splitter.split_documents(webpage_source)
#docs[0].to_json()

In [ ]:
type(docs[0])
#docs)

In [ ]:
# Create an 'embeddings' model
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(api_key=openai_api_key)
#
#embedded_docs = embedding_model.embed_documents(split_docs)

#type(embedded_docs)

In [ ]:
# Create a local vector store
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(split_docs, embedding_model)
type(vector)

In [ ]:
# # Create a model
# from langchain_openai import ChatOpenAI
# print(openai_api_key)
# llm = ChatOpenAI(api_key="openai_api_key")
# llm.invoke("how can langsmith help with testing?")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
#from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# Just local context
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...